In [1]:
from utils.finalyca_store import get_finalyca_scoped_session
db_session = get_finalyca_scoped_session(False)
from fin_models.masters_models import *
from fin_models.transaction_models import *
from sqlalchemy import func, desc, select, and_
import datetime

In [2]:
def temp_fund_stock_tp_json(sql_obj):
    obj = dict()
    obj["AMC_Id"] = sql_obj.AMC_Id
    obj["AMC_Name"] = sql_obj.AMC_Name
    obj["AMC_Logo"] = sql_obj.AMC_Logo
    obj["Fund_Id"] = sql_obj.Fund_Id
    obj["Plan_Id"] = sql_obj.Plan_Id
    obj["Plan_Name"] = sql_obj.Plan_Name
    obj["Product_Id"] = sql_obj.Product_Id
    obj["Product_Name"] = sql_obj.Product_Name
    obj["Classification_Id"] = sql_obj.Classification_Id
    obj["Classification_Name"] = sql_obj.Classification_Name
    obj["Percentage_to_AUM"] = sql_obj.Percentage_to_AUM
    obj["Value_In_Cr"] = sql_obj.Value_In_Cr

    return obj

In [3]:
issuer_id = 329
Classification_Id = None
Product_Id = None
AMC_Id = None

issuer_name = db_session.query(Issuer.Issuer_Name).filter(Issuer.Issuer_Id==issuer_id).scalar()

objs = list()
q = db_session.query(
    AMC.AMC_Id, AMC.AMC_Name, AMC.AMC_Logo, FundStocks.Fund_Id, FundStocks.Plan_Id, FundStocks.Plan_Name, FundStocks.Product_Id, FundStocks.Product_Name, FundStocks.Classification_Id, FundStocks.Classification_Name, func.sum(FundStocks.Percentage_to_AUM).label('Percentage_to_AUM'), func.sum(FundStocks.Value_In_Inr/10000000).label('Value_In_Cr'), func.min(FundStocks.Purchase_Date)
    ).join(MFSecurity, MFSecurity.Fund_Id==FundStocks.Fund_Id).join(AMC, AMC.AMC_Id==MFSecurity.AMC_Id)
q = q.filter(FundStocks.Value_In_Inr> 0.0).filter(FundStocks.Percentage_to_AUM> 0.0)
if Classification_Id:
    q = q.filter(FundStocks.Classification_Id==Classification_Id)
if Product_Id:
    q = q.filter(FundStocks.Product_Id==Product_Id)
if AMC_Id:
    q = q.filter(AMC.AMC_Id==AMC_Id)

q = q.filter(FundStocks.IssuerName==issuer_name)
q = q.group_by(AMC.AMC_Id, AMC.AMC_Name, AMC.AMC_Logo, FundStocks.Fund_Id, FundStocks.Plan_Id, FundStocks.Plan_Name, FundStocks.Product_Id, FundStocks.Product_Name, FundStocks.Classification_Id, FundStocks.Classification_Name, FundStocks.IssuerName)

sql_fund_stocks = q.order_by(desc('Percentage_to_AUM')).all()
for sql_obj in sql_fund_stocks:
    obj = temp_fund_stock_tp_json(sql_obj)
    objs.append(obj)

resp = dict()
resp["total_funds"] = len(sql_fund_stocks)
resp["data"] = objs

resp["highest_weight"] = temp_fund_stock_tp_json(sql_fund_stocks[0])
resp["lowest_weight"] = temp_fund_stock_tp_json(sql_fund_stocks[-1])

sql_fund_stocks = q.order_by(desc('Value_In_Cr')).all()

resp["highest_value"] = temp_fund_stock_tp_json(sql_fund_stocks[0])
resp["lowest_value"] = temp_fund_stock_tp_json(sql_fund_stocks[-1])